In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from dependencies import FCLayer, ActivationLayer, NeuralNetwork, tanh, tanh_derivative, sigmoid, sigmoid_derivative, mse, mse_derivative
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from keras.utils import np_utils

In [4]:
X, Y = load_wine(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
#Normalizing data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(178, 13) (178,)
(142, 13) (36, 13) (142, 3) (36, 3)


In [20]:
network = NeuralNetwork()
network.add(FCLayer(X_train.shape[1], 5))
network.add(ActivationLayer(tanh, tanh_derivative))
network.add(FCLayer(5, 2))
network.add(ActivationLayer(tanh, tanh_derivative))
network.add(FCLayer(2, 3))
network.add(ActivationLayer(sigmoid, sigmoid_derivative))
network.use(mse, mse_derivative)
network.fit(X_train, y_train, epochs=5000, learning_rate=0.1, batch_size=4)
y_pred = np.array(network.predict(X_test))

In [21]:
print(y_test[0:3], y_test.shape)
print(y_pred[0:3], y_pred.shape)
print(accuracy_score(y_test, y_pred.round()))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]] (36, 3)
[[0.0408544  0.04953914 0.90735739]
 [0.94653885 0.01082124 0.04292272]
 [0.0128804  0.95947704 0.04951814]] (36, 3)
0.9722222222222222


In [22]:
clf = MLPClassifier(solver='lbfgs', alpha=0.1,
                                            hidden_layer_sizes=(5,2),
                                            max_iter=1000,activation='tanh')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_test, y_test.shape)
print(y_pred, y_pred.shape)
print(accuracy_score(y_test, y_pred))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]] (36, 3)
[[0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]] (36, 3)
0.9722222222222222


In [3]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    network = NeuralNetwork()
    network.add(FCLayer(X_train.shape[1], config["hidden_layer_sizes"][0]))
    network.add(ActivationLayer(tanh, tanh_derivative))
    network.add(FCLayer(config["hidden_layer_sizes"][0], config["hidden_layer_sizes"][1]))
    network.add(ActivationLayer(tanh, tanh_derivative))
    network.add(FCLayer(config["hidden_layer_sizes"][1], config["output_layer"]))
    network.add(ActivationLayer(sigmoid, sigmoid_derivative))
    network.use(mse, mse_derivative)
    network.fit(X_train, y_train, epochs=config["epochs"], learning_rate=config["alpha"])
    y_pred = np.array(network.predict(X_test))
    reporter(config, mean_accuracy = accuracy_score(y_test, y_pred.round()))
    
# tune.register_trainable("my_func", my_func)
# ray.shutdown()
# ray.init()

algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
# res = tune.run_experiments({
#     "my_experiment": {
#         "run": "my_func",
#         "stop": { "mean_accuracy": 0.6 },
#         "search_alg": algo,
#         "scheduler": scheduler,
#         "num_samples": 10,
#         "config": {
#             "epochs": 1000,
#             "alpha": tune.uniform(0.0001, 10),
#             "hidden_layer_sizes": tune.grid_search([(5,2)]),
#         }
#     }
# })

res = tune.run(
        my_func,
        name="my_exp",
        metric="mean_accuracy",
        stop={"mean_accuracy": 0.6},
        mode="max",
        scheduler=scheduler,
        config={
            "steps": 100,
            "epochs": 1000,
            "alpha": tune.grid_search([0.001, 0.01, 0.1, 1]),
            "hidden_layer_sizes": tune.grid_search([(5,2),(5,3),(5,4)]),
            "output_layer": 3
        },
    )

print("Best hyperparameters found were: ", res.best_config)


c:\Users\Public\anaconda3\lib\site-packages\ray\_private\services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2022-05-15 04:23:45,885	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc,alpha,hidden_layer_sizes
my_func_0bb2b_00000,RUNNING,,0.001,"(5, 2)"
my_func_0bb2b_00001,PENDING,,0.01,"(5, 2)"
my_func_0bb2b_00002,PENDING,,0.1,"(5, 2)"
my_func_0bb2b_00003,PENDING,,1,"(5, 2)"
my_func_0bb2b_00004,PENDING,,0.001,"(5, 3)"
my_func_0bb2b_00005,PENDING,,0.01,"(5, 3)"
my_func_0bb2b_00006,PENDING,,0.1,"(5, 3)"
my_func_0bb2b_00007,PENDING,,1,"(5, 3)"
my_func_0bb2b_00008,PENDING,,0.001,"(5, 4)"
my_func_0bb2b_00009,PENDING,,0.01,"(5, 4)"


Trial name,status,loc,alpha,hidden_layer_sizes
my_func_0bb2b_00000,RUNNING,,0.001,"(5, 2)"
my_func_0bb2b_00001,RUNNING,,0.01,"(5, 2)"
my_func_0bb2b_00002,RUNNING,,0.1,"(5, 2)"
my_func_0bb2b_00003,RUNNING,,1,"(5, 2)"
my_func_0bb2b_00004,RUNNING,,0.001,"(5, 3)"
my_func_0bb2b_00005,RUNNING,,0.01,"(5, 3)"
my_func_0bb2b_00006,RUNNING,,0.1,"(5, 3)"
my_func_0bb2b_00007,RUNNING,,1,"(5, 3)"
my_func_0bb2b_00008,PENDING,,0.001,"(5, 4)"
my_func_0bb2b_00009,PENDING,,0.01,"(5, 4)"


2022-05-15 04:24:39,704	INFO logger.py:697 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (5, 2)}


Result for my_func_0bb2b_00002:
  _metric:
    alpha: 0.1
    epochs: 1000
    hidden_layer_sizes:
    - 5
    - 2
    output_layer: 3
    steps: 100
  date: 2022-05-15_04-24-39
  done: true
  experiment_id: d94f29fd399243c99647fba7a6a8d90c
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.9444444444444444
  node_ip: 192.168.1.105
  pid: 9616
  time_since_restore: 44.56741905212402
  time_this_iter_s: 44.56741905212402
  time_total_s: 44.56741905212402
  timestamp: 1652581479
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0bb2b_00002
  


...

Result for my_func_0bb2b_00011:
  _metric:
    alpha: 1
    epochs: 1000
    hidden_layer_sizes:
    - 5
    - 4
    output_layer: 3
    steps: 100
  date: 2022-05-15_04-25-30
  done: true
  experiment_id: 5551b64d20b84946b48bae66f45936f5
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.9722222222222222
  node_ip: 192.168.1.105
  pid: 13276
  time_since_restore: 29.398839712142944
  time_this_iter_s: 29.398839712142944
  time_total_s: 29.398839712142944
  timestamp: 1652581530
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0bb2b_00011
  


Trial name,status,loc,alpha,hidden_layer_sizes,acc,iter,total time (s)
my_func_0bb2b_00000,TERMINATED,,0.001,"(5, 2)",0.972222,1,55.7104
my_func_0bb2b_00001,TERMINATED,,0.01,"(5, 2)",0.972222,1,56.2151
my_func_0bb2b_00002,TERMINATED,,0.1,"(5, 2)",0.944444,1,44.5674
my_func_0bb2b_00003,TERMINATED,,1,"(5, 2)",0.916667,1,56.617
my_func_0bb2b_00004,TERMINATED,,0.001,"(5, 3)",0.972222,1,45.0097
my_func_0bb2b_00005,TERMINATED,,0.01,"(5, 3)",0.944444,1,44.8093
my_func_0bb2b_00006,TERMINATED,,0.1,"(5, 3)",0.944444,1,55.8754
my_func_0bb2b_00007,TERMINATED,,1,"(5, 3)",0.972222,1,56.6499
my_func_0bb2b_00008,TERMINATED,,0.001,"(5, 4)",0.972222,1,30.5706
my_func_0bb2b_00009,TERMINATED,,0.01,"(5, 4)",0.972222,1,31.1969


Trial name,status,loc,alpha,hidden_layer_sizes,acc,iter,total time (s)
my_func_0bb2b_00000,TERMINATED,,0.001,"(5, 2)",0.972222,1,55.7104
my_func_0bb2b_00001,TERMINATED,,0.01,"(5, 2)",0.972222,1,56.2151
my_func_0bb2b_00002,TERMINATED,,0.1,"(5, 2)",0.944444,1,44.5674
my_func_0bb2b_00003,TERMINATED,,1,"(5, 2)",0.916667,1,56.617
my_func_0bb2b_00004,TERMINATED,,0.001,"(5, 3)",0.972222,1,45.0097
my_func_0bb2b_00005,TERMINATED,,0.01,"(5, 3)",0.944444,1,44.8093
my_func_0bb2b_00006,TERMINATED,,0.1,"(5, 3)",0.944444,1,55.8754
my_func_0bb2b_00007,TERMINATED,,1,"(5, 3)",0.972222,1,56.6499
my_func_0bb2b_00008,TERMINATED,,0.001,"(5, 4)",0.972222,1,30.5706
my_func_0bb2b_00009,TERMINATED,,0.01,"(5, 4)",0.972222,1,31.1969


(pid=13276) Windows fatal exception: access violation
(pid=13276) 
2022-05-15 04:25:30,408	INFO tune.py:561 -- Total run time: 105.10 seconds (104.70 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 100, 'epochs': 1000, 'alpha': 0.001, 'hidden_layer_sizes': (5, 2), 'output_layer': 3}


In [19]:
#print("Best hyperparameters found were: ", res.best_result)
#print(res.results)
results = {k: v for k, v in sorted(res.results.items(), key=lambda item: (item[1]["mean_accuracy"], item[1]["time_this_iter_s"]), reverse=True)}
print(list(results.values())[0])

{'mean_accuracy': 0.9722222222222222, '_metric': {'steps': 100, 'epochs': 1000, 'alpha': 1, 'hidden_layer_sizes': (5, 3), 'output_layer': 3}, 'time_this_iter_s': 56.64989185333252, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 1, 'experiment_id': '2575815716dc4ca7bc345111d186e399', 'date': '2022-05-15_04-24-51', 'timestamp': 1652581491, 'time_total_s': 56.64989185333252, 'pid': 15248, 'hostname': 'Ghallab', 'node_ip': '192.168.1.105', 'config': {'steps': 100, 'epochs': 1000, 'alpha': 1, 'hidden_layer_sizes': (5, 3), 'output_layer': 3}, 'time_since_restore': 56.64989185333252, 'timesteps_since_restore': 0, 'iterations_since_restore': 1, 'trial_id': '0bb2b_00007', 'experiment_tag': '7_alpha=1,hidden_layer_sizes=(5, 3)'}


(pid=None) [2022-05-15 05:30:53,305 C 3580 13036] redis_client.cc:87:  Check failed: under_retry_limit Expected 1 Redis shard addresses, found 2
(pid=None) *** StackTrace Information ***
(pid=None) 
(pid=14084) 2022-05-15 05:31:15,708	ERROR function_runner.py:266 -- Runner Thread raised error.
(pid=14084) Traceback (most recent call last):
(pid=14084)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 260, in run
(pid=14084)     self._entrypoint()
(pid=14084)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 328, in entrypoint
(pid=14084)     return self._trainable_func(self.config, self._status_reporter,
(pid=14084)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 594, in _trainable_func
(pid=14084)     output = fn()
(pid=14084)   File "<ipython-input-10-2eb213c7e56b>", line 53, in __network_generator
(pid=14084)   File "<ipython-input-10-2eb213c7e56b>", line 36, in __create_